#  Frequent Itemset Mining: Apriori Alternatives

In this notebook, we will apply **apriori**, **FP-Growth**, and **maximal frequent itemset** methods on the same retail dataset that we explored in M3. 

 ### Import required Libraries

In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax
import matplotlib.pyplot as plt
%matplotlib inline

### T1: Data Loading

The data is located here: `/dsa/data/DSA-8410/association-mining/retail_dataset.csv`


In [3]:
df = pd.read_csv('/dsa/data/DSA-8410/association-mining/retail_dataset.csv')
df.head(5)

0       1     2       3       4       5       6
0   Bread    Wine  Eggs    Meat  Cheese  Pencil  Diaper
1   Bread  Cheese  Meat  Diaper    Wine    Milk  Pencil
2  Cheese    Meat  Eggs    Milk    Wine     NaN     NaN
3  Cheese    Meat  Eggs    Milk    Wine     NaN     NaN
4    Meat  Pencil  Wine     NaN     NaN     NaN     NaN

### T2: Show the number of transactions and unique items

In [4]:
print(f"Num of transactions = {df.shape[0]}")
print(f"Maximum num of items per transaction = {df.shape[1]}")

Num of transactions = 315
Maximum num of items per transaction = 7


Let’s find out the unique items in this dataset. 

In [5]:
set(df.values.flatten())

{'Bagel',
 'Bread',
 'Cheese',
 'Diaper',
 'Eggs',
 'Meat',
 'Milk',
 'Pencil',
 'Wine',
 nan}

### T3: Transform the dataset to a binary incidence matrix for applying itemset mining methods

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

trans_data = []
for indx, row in df.iterrows():
    trans_data.append(row.dropna().values)

mlb = MultiLabelBinarizer()
data = mlb.fit_transform(trans_data)
mlb.classes_

array(['Bagel', 'Bread', 'Cheese', 'Diaper', 'Eggs', 'Meat', 'Milk',
       'Pencil', 'Wine'], dtype=object)

In [9]:
trans_data_enc = pd.DataFrame(data, columns=mlb.classes_)
trans_data_enc.head()

Bagel  Bread  Cheese  Diaper  Eggs  Meat  Milk  Pencil  Wine
0      0      1       1       1     1     1     0       1     1
1      0      1       1       1     0     1     1       1     1
2      0      0       1       0     1     1     1       0     1
3      0      0       1       0     1     1     1       0     1
4      0      0       0       0     0     1     0       1     1

### T4.1: Indentify Frequent Patterns with Apriori Method. Use min_support = 0.2. Show all frequent 3-itemsets. 

In [12]:
freq_items = apriori(trans_data_enc, min_support=0.2, use_colnames=True, verbose=1)

Processing 4 combinations | Sampling itemset size 4


In [13]:
freq_items.shape

(33, 2)

In [17]:
freq_items = freq_items.reindex(columns=['itemsets','support'])
freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))

In [18]:
freq_items[freq_items['length']>2]

itemsets   support  length
31  (Meat, Eggs, Cheese)  0.215873       3
32  (Meat, Milk, Cheese)  0.203175       3

### T4.2: Generate Association Rules from Frequent Itemsets. Show the top-5 rules with high conviction. 

In [20]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.6)
rules.head()

antecedents consequents  antecedent support  consequent support   support  \
0     (Bagel)     (Bread)            0.425397            0.504762  0.279365   
1      (Eggs)    (Cheese)            0.438095            0.501587  0.298413   
2      (Meat)    (Cheese)            0.476190            0.501587  0.323810   
3    (Cheese)      (Meat)            0.501587            0.476190  0.323810   
4      (Milk)    (Cheese)            0.501587            0.501587  0.304762   

   confidence      lift  leverage  conviction  
0    0.656716  1.301042  0.064641    1.442650  
1    0.681159  1.358008  0.078670    1.563203  
2    0.680000  1.355696  0.084958    1.557540  
3    0.645570  1.355696  0.084958    1.477891  
4    0.607595  1.211344  0.053172    1.270148

In [21]:
rules.sort_values(by=['conviction'], ascending=False).head()

antecedents consequents  antecedent support  consequent support  \
11    (Meat, Milk)    (Cheese)            0.244444            0.501587   
8     (Meat, Eggs)    (Cheese)            0.266667            0.501587   
10  (Eggs, Cheese)      (Meat)            0.298413            0.476190   
9   (Meat, Cheese)      (Eggs)            0.323810            0.438095   
13  (Milk, Cheese)      (Meat)            0.304762            0.476190   

     support  confidence      lift  leverage  conviction  
11  0.203175    0.831169  1.657077  0.080564    2.952137  
8   0.215873    0.809524  1.613924  0.082116    2.616667  
10  0.215873    0.723404  1.519149  0.073772    1.893773  
9   0.215873    0.666667  1.521739  0.074014    1.685714  
13  0.203175    0.666667  1.400000  0.058050    1.571429

### T5: Infere frequent itemsets with FP-Growth. Use min support = 0.2. 

In [22]:
fpgrowth(trans_data_enc, min_support=0.2, use_colnames=True)

support              itemsets
0   0.504762               (Bread)
1   0.501587              (Cheese)
2   0.476190                (Meat)
3   0.438095                (Wine)
4   0.438095                (Eggs)
5   0.406349              (Diaper)
6   0.361905              (Pencil)
7   0.501587                (Milk)
8   0.425397               (Bagel)
9   0.238095       (Cheese, Bread)
10  0.304762        (Milk, Cheese)
11  0.323810        (Meat, Cheese)
12  0.206349         (Meat, Bread)
13  0.244444          (Meat, Milk)
14  0.203175  (Meat, Milk, Cheese)
15  0.269841        (Wine, Cheese)
16  0.250794          (Meat, Wine)
17  0.244444         (Wine, Bread)
18  0.219048          (Milk, Wine)
19  0.298413        (Eggs, Cheese)
20  0.266667          (Meat, Eggs)
21  0.241270          (Wine, Eggs)
22  0.244444          (Milk, Eggs)
23  0.215873  (Meat, Eggs, Cheese)
24  0.234921        (Diaper, Wine)
25  0.231746       (Diaper, Bread)
26  0.200000      (Diaper, Cheese)
27  0.200000        (Pencil, Wine)
28  0.200000      (Pencil, Cheese)
29  0.200000       (Pencil, Bread)
30  0.279365         (Milk, Bread)
31  0.279365        (Bagel, Bread)
32  0.225397         (Milk, Bagel)

### T6. Show the maximal frequent itemsets for min support = 0.2 

In [23]:
max_patterns = fpmax(trans_data_enc, min_support=0.2, use_colnames=True)

In [24]:
# for readability
max_patterns = max_patterns.reindex(columns=['itemsets','support'])
max_patterns['length'] = max_patterns['itemsets'].apply(lambda x: len(x))

In [25]:
print(f"Total number of maximal frequent patterns = {max_patterns.shape[0]}")
max_patterns

Total number of maximal frequent patterns = 19


itemsets   support  length
0        (Pencil, Bread)  0.200000       2
1       (Pencil, Cheese)  0.200000       2
2         (Pencil, Wine)  0.200000       2
3       (Diaper, Cheese)  0.200000       2
4        (Diaper, Bread)  0.231746       2
5         (Diaper, Wine)  0.234921       2
6          (Milk, Bagel)  0.225397       2
7         (Bagel, Bread)  0.279365       2
8           (Wine, Eggs)  0.241270       2
9           (Milk, Eggs)  0.244444       2
10  (Meat, Eggs, Cheese)  0.215873       3
11          (Milk, Wine)  0.219048       2
12         (Wine, Bread)  0.244444       2
13          (Meat, Wine)  0.250794       2
14        (Wine, Cheese)  0.269841       2
15         (Meat, Bread)  0.206349       2
16  (Meat, Milk, Cheese)  0.203175       3
17       (Cheese, Bread)  0.238095       2
18         (Milk, Bread)  0.279365       2

# Save your notebook, then `File > Close and Halt`